In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *

In [2]:
import transformers
transformers.__version__

'3.4.0'

In [3]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [4]:
positive_train_path = '/home/advaitmb/.fastai/data/imdb/train/pos/'
positive_train_files = get_text_files(positive_train_path)

negative_train_path = '/home/advaitmb/.fastai/data/imdb/train/neg/'
negative_train_files = get_text_files(negative_train_path)


positive_test_path = '/home/advaitmb/.fastai/data/imdb/test/pos/'
positive_test_files = get_text_files(positive_test_path)

negative_test_path = '/home/advaitmb/.fastai/data/imdb/test/neg/'
negative_test_files = get_text_files(negative_test_path)

In [5]:
def files_to_list(files):
    ls = []
    for file in files:
        with open(file, 'r') as f:
            ls.append(f.read())
    return ls


In [6]:
positive_train_text_list = files_to_list(positive_train_files)
negative_train_text_list = files_to_list(negative_train_files)
positive_test_text_list = files_to_list(positive_test_files)
negative_test_text_list = files_to_list(negative_test_files)

In [7]:
import re

def clean_html(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [8]:
def clean_newlines(raw_text):
    return raw_text.replace('\n', '')

In [9]:
for i in range(len(positive_train_text_list)):
    positive_train_text_list[i] = clean_newlines(clean_html(positive_train_text_list[i]))
    
for i in range(len(negative_train_text_list)):
    negative_train_text_list[i] = clean_newlines(clean_html(negative_train_text_list[i]))
    
for i in range(len(positive_test_text_list)):
    positive_test_text_list[i] = clean_newlines(clean_html(positive_test_text_list[i]))
    
for i in range(len(negative_test_text_list)):
    negative_test_text_list[i] = clean_newlines(clean_html(negative_test_text_list[i]))

In [10]:
# with open('positive_train_text', 'w') as f:
#     for text in positive_train_text_list:
#         f.write(text + '\n')
        
# with open('negative_train_text', 'w') as f:
#     for text in negative_train_text_list:
#         f.write(text + '\n')
        
with open('positive_test_text', 'w') as f:
    for text in positive_test_text_list:
        f.write(text + '\n')
        
with open('negative_test_text', 'w') as f:
    for text in negative_test_text_list:
        f.write(text + '\n')

In [15]:
with open('positive_train_text', 'r') as f:
    print(f.read()[:5000])

Steven Speilberg's adaptation of Alice Walkers popular novel is not without its share of controversy. When first released members of the black community criticised its treatment of black men, while others questioned why a white man was directing this film about black women.This is the story of a young black woman named Celie, growing up in rural America after the turn of the century. She has two children by her abusive father which are snatched from her arms at birth. Her only solace in her miserable life comes from her sister.Celie (played in later years by newcomer Whoopie Goldberg) is married off to an abusive husband (Danny Glover). The husband is humiliated by the sister and so she is quickly removed from Celie's life.The story is often heartbreaking as Celie keeps up hope that she may one day be reunited with her sister and with her children. Throughout her life she meets an assortment of characters, including Sophia, a tough as nails wife to her step son, and Shug, a loud and lu

In [6]:
train_path = 'positive_train_text'
test_path = 'positive_test_text'

In [7]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [8]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="./gpt2-imdb-positive-sentiment", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

/anaconda/envs/fastai2/lib/python3.8/site-packages/transformers/trainer.py:259: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  warnings.warn(


In [9]:
trainer.train()

Step,Training Loss
500,4.031479
1000,3.950646
1500,3.899207
2000,3.886410
2500,3.879925
3000,3.856635
3500,3.827273
4000,3.713354
4500,3.694951
5000,3.695477


TrainOutput(global_step=10749, training_loss=3.7284249552283932)

In [11]:
trainer.evaluate?

In [14]:
trainer.compute_metrics

In [30]:
prompt = "this movie is"
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()

In [31]:
beam_outputs = trainer.model.generate(
    inp, 
    max_length=20, 
    num_beams=10, 
    repeatition_penalty=2., 
    num_return_sequences=10, 
    early_stopping=True,
    do_sample=True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [32]:
for i, beam_output in enumerate(beam_outputs):
    output = tokenizer.decode(beam_output.cpu().numpy(), skip_special_tokens=True)
#     sentiment = sentiment_model.forward(sentiment_tokenizer.encode(prompt[len(prompt):], return_tensors="pt"))
    
    print("{}: \n{}".format(i, output))
    

0: 
this movie is the best I have seen in a long time.
This is one of the best
1: 
this movie is one of the best I've seen in a long time. I don't know if
2: 
this movie is not for everyone, but it is a great way to spend time with a family.
3: 
this movie is a must-see.
I saw this movie for the first time when I was
4: 
this movie is a must-see for all sci-fi fans.
This is one of the
5: 
this movie is a must-see for all ages.
This is one of the best movies I
6: 
this movie is one of the best movies I've seen in a long time. If you don't
7: 
this movie is one of the best I've seen in a long time. I've seen a lot
8: 
this movie is one of the best movies I have seen in a long time. It is one of
9: 
this movie is the best movie I've seen in a long time. I can't wait to see


In [ ]:
trainer.evaluate()

In [ ]:
max_length = model.config.n_positions
stride = 512

lls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len

    lls.append(log_likelihood)

ppl = torch.exp(torch.stack(lls).sum() / end_loc)